In [1]:
import pandas as pd
from faker import Faker
import datetime
import numpy as np

from joblib import Parallel, delayed
from alive_progress import alive_bar
from thefuzz import fuzz, process
from tqdm.notebook import tqdm_notebook

In [20]:
fake = Faker()

  0%|          | 0/100 [00:00<?, ?it/s]


In [16]:
def generate_synthetic_data(n, column_types):
    """
    Generate synthetic data using Faker library based on specified column types.

    Parameters:
    n: integer number of synthetic data rows to generate.
    column_types (dict): Dictionary where keys are column names and values are Faker providers.

    Returns:
    pd.DataFrame: DataFrame containing synthetic data.
    """

    fake = Faker()
    data = {col: [getattr(fake, col_type)() for _ in range(n)] for col, col_type in column_types.items()}
    df = pd.DataFrame(data)

    return df

def add_variance(df, num_records_to_vary, column_types):
    # Randomly choose rows and columns to vary
    rows_to_vary = np.random.randint(0, len(df), num_records_to_vary)
    modified_df = df.loc[rows_to_vary]
    cols_to_vary = np.random.choice(df.columns, num_records_to_vary, replace=True)

    # Add variance to randomly sampled records
    for i in range(num_records_to_vary):
        row_idx = rows_to_vary[i]
        col = cols_to_vary[i]
        if column_types[col] == 'date_of_birth':
            modified_df.at[row_idx, col] = modified_df.at[row_idx, col] + pd.Timedelta(days=int(np.random.normal(0, 10)))
        elif column_types[col] == 'float64':
            modified_df.at[row_idx, col] = modified_df.at[row_idx, col] + np.random.normal(0, 0.1)
        else:
            modified_df.at[row_idx, col] = str(modified_df.at[row_idx, col]) + str(np.random.normal(0, 0.1))

    new_df = generate_synthetic_data(len(df)-num_records_to_vary, column_types)
    new_df = pd.concat([new_df, modified_df], axis=0)

    return new_df.reset_index(drop=True)

def match_rows(row1, df2, threshold=70):
    matches = []
    with tqdm_notebook(total=len(df2)) as pbar:
        for _, row2 in df2.iterrows():
            # Calculate the fuzzy match score for first name, last name, and birthday
            first_name_score = fuzz.token_sort_ratio(row1['First Name'], row2['First Name'])
            last_name_score = fuzz.token_sort_ratio(row1['Last Name'], row2['Last Name'])
            birthday_score = fuzz.token_sort_ratio(row1['date_of_birth'], row2['date_of_birth'])
    
            # Check if all three scores are above the threshold
            if first_name_score >= threshold and last_name_score >= threshold and birthday_score >= threshold:
                # Add the matched row to the list
                matches.append(row2)

            pbar.update(1)
            
    return matches

def deidentify_data(df):
    df['personal_id'] = [fake.uuid4() for _ in range(len(df))]
    df['Date of Birth'] = pd.to_datetime(df['date_of_birth'])
    df['Month/Year of Birth'] = df['date_of_birth'].apply(lambda date: date.strftime('%m-%Y'))
    df = df.drop(columns=['Last Name','First Name','Address'])
    return df


In [17]:
column_types = {
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'date_of_birth': 'date_of_birth',
    'job': 'job',
    'Address': 'address',
    'email': 'email',
    'phone_number': 'phone_number',
    'email': 'email'
}

synthetic_df = generate_synthetic_data(100, column_types)


synthetic_df_with_variance = add_variance(synthetic_df, 10, column_types)

synthetic_df_with_variance.sort_values(by='First Name')

,First Name,Last Name,date_of_birth,job,Address,email,phone_number
50,Adam,Blankenship,1913-09-20,Holiday representative,"319 Dalton Alley Suite 269\nStephaniehaven, WI...",cwolfe@example.net,922.313.3826
3,Alexander,Erickson,1973-06-04,Engineering geologist,"PSC 3617, Box 3701\nAPO AP 48005",matthew43@example.com,1028032807
36,Alicia,Jensen,1965-04-13,Solicitor,"059 Mary Flat\nHarringtonton, MO 28212",debbie24@example.net,419-905-7726
80,Andrew,White,1931-10-15,"Doctor, hospital","567 Martin Ford Apt. 093\nCurtistown, FM 45118",stephaniemartinez@example.org,001-517-480-4348x60250
24,Andrew,Smith,1958-12-11,Human resources officer,"1754 Kevin Vista Apt. 605\nNorth Jessica, SC 4...",fcampbell@example.net,580-752-7814x073
...,...,...,...,...,...,...,...
85,Trevor,Cole,1982-08-12,Therapeutic radiographer,USNS Cole\nFPO AP 79593,anthonystone@example.org,001-682-360-1444
65,Victoria,Rivers,1986-01-21,Pension scheme manager,"844 Kane Mission\nNorth Rachel, CA 14441",paynetyler@example.com,+1-980-721-6781x31372
38,Wendy,Roth,1946-10-30,Land/geomatics surveyor,"820 Case Crossing Suite 224\nNorth Paul, IL 66653",rortiz@example.org,+1-660-657-6510x86675
58,William,Hernandez,1987-12-11,Automotive engineer,"76371 Molina Land Suite 742\nDennisport, DC 06649",sherryvazquez@example.net,473.667.1620x8938


In [18]:
synthetic_df.sort_values(by='First Name')

,First Name,Last Name,date_of_birth,job,Address,email,phone_number
97,Adam,Allen,1964-07-09,IT trainer,"25298 Larson Drives\nLake Ashley, VA 22462",kelli75@example.org,109.207.6406
9,Adrian,Burnett,1953-09-10,"Education officer, environmental",USNS Cox\nFPO AE 48555,yrivera@example.com,+1-941-902-0949x6381
76,Alvin,Guerrero,1962-02-17,Clinical embryologist,"982 Beasley Pine\nCordovashire, NJ 76422",klee@example.net,894.661.3823x69507
29,Andrew,Moore,1954-03-30,Teaching laboratory technician,"28691 Knight Loaf\nPort Steven, ND 39066",bryantdeanna@example.org,713-983-1724x029
64,Angela,Tucker,1915-01-06,Art gallery manager,"151 Robert Crossroad\nAndersonmouth, SC 51058",mgalloway@example.net,001-192-925-8934x799
...,...,...,...,...,...,...,...
22,Walter,Dixon,1952-07-22,Fitness centre manager,"PSC 1247, Box 8104\nAPO AA 50145",jeremymalone@example.org,4317401487
86,William,Henderson,1957-10-02,Statistician,"779 Murray Plaza\nLeemouth, MP 21616",wtate@example.org,182-352-3215
36,William,Parker,1956-09-23,Building control surveyor,"830 Miller Island Suite 269\nNew Ronaldton, CO...",carrieroberts@example.net,001-962-807-5911
3,William,Berry,1934-01-02,Psychiatric nurse,"7138 Jeffrey Port Suite 045\nFostertown, TX 17999",scottpamela@example.org,(461)202-1914x26301


In [19]:
df1 = synthetic_df_with_variance
df2 = synthetic_df

matched_rows = Parallel(n_jobs=-1)(delayed(match_rows)(row1, df2) for _, row1 in df1.iterrows())
# Flatten the list of lists
matched_rows = [row for sublist in matched_rows for row in sublist]

# Create a new dataframe containing the matched rows
matched_df = pd.DataFrame(matched_rows, columns=df1.columns.to_list())


In [7]:
matched_df

,First Name,Last Name,date_of_birth,job,Address,email,phone_number
29,Andrew,Thompson,1919-10-01,"Development worker, international aid","20152 Foster Glens Apt. 957\nNew Teresaville, ...",codyrogers@example.org,001-704-709-9805x12064
61,Daniel,Sims,2009-01-24,"Engineer, energy","PSC 1517, Box 6002\nAPO AA 96537",mary36@example.com,094.082.2926x92995
35,Diana,Norman,1978-06-11,Trading standards officer,"PSC 7737, Box 5769\nAPO AP 51332",prestonjason@example.org,+1-244-967-1539x52821
33,Kathleen,Martinez,1917-02-01,Press sub,"771 James Orchard\nNorth Robertaport, AL 16625",kathleenmccoy@example.org,813-813-9506
90,Julie,Moreno,1983-06-16,Bookseller,"137 Jackson Radial\nSmithton, FL 04524",ricepaul@example.org,(444)908-6540x13751


In [8]:
def fuzzy_match_row(row1, df2, threshold=70):
    matches = []
    for _, row2 in df2.iterrows():
        first_name_score = fuzz.token_sort_ratio(row1['First Name'], row2['First Name'])
        if first_name_score >= threshold:
            row2['first_name_score'] = first_name_score
            last_name_score = fuzz.token_sort_ratio(row1['Last Name'], row2['Last Name'])
            if last_name_score >= threshold:
                row2['surname_score'] = last_name_score
                birthday_score = fuzz.token_sort_ratio(row1['date_of_birth'], row2['date_of_birth'])
                if birthday_score >= threshold:
                    row2['birthday_score'] = birthday_score
                    matches.append(row2)
        
    return matches

In [9]:
batch_size = 10
batch_indices = range(0, len(df2), batch_size)
batches = [df2.iloc[i:i+batch_size] for i in batch_indices]

matched_rows = []  

for _, row1 in tqdm_notebook(df1.iterrows(), total=len(df1), desc="Processing"):    
    # Perform fuzzy matching in parallel using joblib
    batch_matches = Parallel(n_jobs=-1)(
        delayed(fuzzy_match_row)(row1, batch_df) for batch_df in batches
    )

    # Flatten the list of lists
    matched_rows.extend([row for sublist in batch_matches for row in sublist])

# Create a new dataframe containing the matched rows
columns = df1.columns.tolist() + ['first_name_score', 'surname_score', 'birthday_score']

matched_df = pd.DataFrame(matched_rows, columns=columns)
matched_df

Processing:   0%|          | 0/100 [00:00<?, ?it/s]

,First Name,Last Name,date_of_birth,job,Address,email,phone_number,first_name_score,surname_score,birthday_score
29,Andrew,Thompson,1919-10-01,"Development worker, international aid","20152 Foster Glens Apt. 957\nNew Teresaville, ...",codyrogers@example.org,001-704-709-9805x12064,100,100,100
61,Daniel,Sims,2009-01-24,"Engineer, energy","PSC 1517, Box 6002\nAPO AA 96537",mary36@example.com,094.082.2926x92995,100,100,100
35,Diana,Norman,1978-06-11,Trading standards officer,"PSC 7737, Box 5769\nAPO AP 51332",prestonjason@example.org,+1-244-967-1539x52821,100,100,100
33,Kathleen,Martinez,1917-02-01,Press sub,"771 James Orchard\nNorth Robertaport, AL 16625",kathleenmccoy@example.org,813-813-9506,100,100,100
90,Julie,Moreno,1983-06-16,Bookseller,"137 Jackson Radial\nSmithton, FL 04524",ricepaul@example.org,(444)908-6540x13751,100,100,100


In [10]:
matched_df

,First Name,Last Name,date_of_birth,job,Address,email,phone_number,first_name_score,surname_score,birthday_score
29,Andrew,Thompson,1919-10-01,"Development worker, international aid","20152 Foster Glens Apt. 957\nNew Teresaville, ...",codyrogers@example.org,001-704-709-9805x12064,100,100,100
61,Daniel,Sims,2009-01-24,"Engineer, energy","PSC 1517, Box 6002\nAPO AA 96537",mary36@example.com,094.082.2926x92995,100,100,100
35,Diana,Norman,1978-06-11,Trading standards officer,"PSC 7737, Box 5769\nAPO AP 51332",prestonjason@example.org,+1-244-967-1539x52821,100,100,100
33,Kathleen,Martinez,1917-02-01,Press sub,"771 James Orchard\nNorth Robertaport, AL 16625",kathleenmccoy@example.org,813-813-9506,100,100,100
90,Julie,Moreno,1983-06-16,Bookseller,"137 Jackson Radial\nSmithton, FL 04524",ricepaul@example.org,(444)908-6540x13751,100,100,100


In [11]:
deIdentified_df = deidentify_data(matched_df)
deIdentified_df

,date_of_birth,job,email,phone_number,first_name_score,surname_score,birthday_score,personal_id,Date of Birth,Month/Year of Birth
29,1919-10-01,"Development worker, international aid",codyrogers@example.org,001-704-709-9805x12064,100,100,100,1ff84d8a-c27c-44f7-8ae8-43240f52d5a7,1919-10-01,10-1919
61,2009-01-24,"Engineer, energy",mary36@example.com,094.082.2926x92995,100,100,100,91cc7858-e4a7-43b4-b2dd-071af9acdf1a,2009-01-24,01-2009
35,1978-06-11,Trading standards officer,prestonjason@example.org,+1-244-967-1539x52821,100,100,100,9528aceb-5f48-459a-a9f6-49b5480da04a,1978-06-11,06-1978
33,1917-02-01,Press sub,kathleenmccoy@example.org,813-813-9506,100,100,100,1d284d1c-936b-4b55-990e-869d37dbefc9,1917-02-01,02-1917
90,1983-06-16,Bookseller,ricepaul@example.org,(444)908-6540x13751,100,100,100,c87936b6-3367-4137-884a-04d47eaf8676,1983-06-16,06-1983
